#**TABLEZAP CUSTOMER CHURN PREDICTION CASE STUDY**

**FIRST OF ALL LET'S INSTALL THE REQUIRED LIBRARY THAT IS PYSPARK FOR THE REQUIRED TABLE.**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=a48b8492f497b6737f66938fff8c720a174f689d64c8190a4b73f4f06a4f562e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


**LET'S WRITE THE CODE FOR CREATING THE SPARK SESSION, AND ALSO FOR IMPORTING THE REQUIRED FUNCTIONS SUCH AS COUNT, AVERAGE ETC.**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, datediff

# Create a SparkSession
spark = SparkSession.builder.appName("CustomerMetrics").getOrCreate()

**LET'S READ ALL THE CSV FILES WHICH ARE GIVEN IN ORDER TO GAIN ALL THE INSIGHTS FROM THE DATA AND ALSO THESE ARE REQUIRED TO MAKE THE NECESSARY COLUMNS FOR THE TABLE THAT WILL BE FURTHER USED FOR THE ANALYSIS AND ALSO FOR THE CHURN PREDICTION.**

In [4]:
# Read the tables into Spark DataFrames
customer_data = spark.read.format("csv").option("header", "true").load("/content/customer_data.csv")
order_history = spark.read.format("csv").option("header", "true").load("/content/order_history.csv")
customer_interactions = spark.read.format("csv").option("header", "true").load("/content/customer_interactions.csv")
loyalty_program = spark.read.format("csv").option("header", "true").load("/content/loyalty_program.csv")

**NOW AFTER READING ALL THE CSV FILES LET'S MOVE ON TO OUR NEXT AND THE MOST IMPORTANT STEPS THAT IS MAKING OF THE CHURN PREDICTION TABLE**

**1. SO THE FIRST COLUMN THAT WE ARE GOING TO CREATE IS THE "FREQUENCY OF VISITS", THIS COLUMN WILL SHOW THAT, HOW MANY NUMBER OF TIMES DOES A PARTICULAR CUSTOMER ID VISITS TO THE RESTAURANT.** 

In [5]:
# Calculate frequency of visits
frequency_of_visits = order_history.groupBy("customer_id").agg(count("*").alias("frequency_of_visits"))

**2. NOW LET'S MOVE ON TO OUR NEXT COLUMN, WHICH IS "AVERAGE ORDER VALUE", THIS COLUMN WILL SHOW THAT,ON AN AVERAGE HOW MUCH ORDERS ARE PLACED BY A PARTICULAR CUSTOMER ID.**

In [6]:
# Calculate average order value
average_order_value = order_history.groupBy("customer_id").agg(avg("order_total").alias("average_order_value"))

**3. NOW LET'S WRITE THE CODE FOR THE NEXT COLUMN WHICH IS "TIME SPENT PER VISIT", THIS COLUMN WILL INTREPRET THAT HOW MUCH TIME DOES A PARTICULAR CUSTOMER ID SPENT IN THE RESTAURANT.**

In [7]:
# Calculate time spent per visit
time_spent_per_visit = order_history.alias("o").join(
    order_history.alias("o2"),
    (col("o.customer_id") == col("o2.customer_id")) & (col("o.order_date") > col("o2.order_date")),
    "left"
).groupBy("o.customer_id", "o.order_date").agg(
    avg(datediff(col("o.order_date"), col("o2.order_date"))).alias("time_spent_per_visit")
).groupBy("customer_id").agg(avg("time_spent_per_visit").alias("time_spent_per_visit"))

**4. NOW LET'S CALCULATE OUR LAST COLUMN THAT IS "FEEDBACK SENTIMENT", THIS WILL GIVE US THE INFORMATION ABOUT THE AVERAGE FEEDBACK RATING BY A PARTICULAR CUSTOMER ID.**

In [8]:
# Calculate feedback sentiment
feedback_sentiment = order_history.where(col("feedback_rating").isNotNull()).groupBy("customer_id").agg(avg("feedback_rating").alias("feedback_sentiment"))

**SO NOW AS WE HAVE CALCULATED OUR ALL THE FOUR REQUIRED COLUMNS, LET'S MERGE THEM INTO A TABLE.**

In [9]:
# Join all the metrics together
customer_metrics = customer_data.select("customer_id", "customer_name").join(
    frequency_of_visits, "customer_id"
).join(
    average_order_value, "customer_id"
).join(
    time_spent_per_visit, "customer_id"
).join(
    feedback_sentiment, "customer_id"
)

In [10]:
customer_metrics.show()

+-----------+-----------------+-------------------+-------------------+--------------------+------------------+
|customer_id|    customer_name|frequency_of_visits|average_order_value|time_spent_per_visit|feedback_sentiment|
+-----------+-----------------+-------------------+-------------------+--------------------+------------------+
|        691|    Matthew Moore|                  8|           2406.625|  134.89897959183673|             1.875|
|        829|   Mary Rodriguez|                  4|            2671.75|   302.9444444444444|              3.25|
|        296|       Mario Rush|                  5|             2180.0|  238.14583333333334|               3.0|
|        675|     Kristy Huynh|                  2|             2905.0|               410.0|               3.0|
|        467|    Michael Sloan|                  4|             1998.5|   245.1111111111111|               4.0|
|        944|     Keith Harmon|                  4|             2755.5|               177.0|            

**LET'S STORE THE ABOVE TABLE IN CSV FORMAT.**

In [11]:
# Convert Spark DataFrame to Pandas DataFrame
customer_metrics_pandas = customer_metrics.toPandas()

# Save Pandas DataFrame to an Excel file
customer_metrics_pandas.to_csv("customer_metrics.csv", index=False)

# Stop the SparkSession
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7fdbd8ea25f0>>